<p align="left">
  <img src="https://raw.githubusercontent.com/python35/IINTS-SDK/main/img/iints_logo.png" width="160">
</p>
# Data Registry & Real-World Import
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/python35/IINTS-SDK/blob/main/examples/notebooks/08_Data_Registry_and_Import.ipynb)

**Goal:** discover official datasets, fetch the bundled sample set, and import it into a runnable scenario.

**You will learn:**
- List datasets in the official registry
- Fetch the bundled sample dataset (offline)
- Convert CGM CSV into an IINTS scenario
- Run a short simulation from imported data


In [ ]:
from __future__ import annotations
from pathlib import Path
import os
import sys
import subprocess


def _find_repo_root() -> Path | None:
    for root in [Path.cwd(), *Path.cwd().parents]:
        if (root / "pyproject.toml").exists() and (root / "src").exists():
            return root
    return None

repo_root = _find_repo_root()
if repo_root is None:
    try:
        import google.colab  # type: ignore
        in_colab = True
    except Exception:
        in_colab = False

    if not in_colab:
        raise RuntimeError("Run this notebook inside the IINTS-SDK repo or on Colab.")

    if not Path("IINTS-SDK").exists():
        subprocess.check_call(["git", "clone", "https://github.com/python35/IINTS-SDK.git"])
    repo_root = Path("IINTS-SDK").resolve()

os.chdir(repo_root)
sys.path.insert(0, str(repo_root / "src"))
print("Repo root:", repo_root)


## Step 1: List datasets


In [ ]:
from iints.data import load_dataset_registry

registry = load_dataset_registry()
[{"id": d["id"], "name": d["name"], "access": d["access"]} for d in registry]


## Step 2: Fetch the bundled sample dataset (offline)


In [ ]:
from pathlib import Path
from iints.data import fetch_dataset

output_dir = Path("data_packs/sample")
paths = fetch_dataset("sample", output_dir=output_dir, extract=False)
paths


## Step 3: Convert CSV to scenario


In [ ]:
from iints.data import scenario_from_csv

sample_csv = paths[0]
result = scenario_from_csv(sample_csv, scenario_name="Sample CGM")
result.scenario


## Step 4: Run a short simulation from imported data


In [ ]:
import iints
from iints.core.algorithms.fixed_basal_bolus import FixedBasalBolus
from iints.validation import load_patient_config_by_name

patient_config = load_patient_config_by_name("clinic_safe_baseline").model_dump()
algorithm = FixedBasalBolus(settings={"fixed_basal_rate": 0.4, "carb_ratio": 12.0})

outputs = iints.run_simulation(
    algorithm=algorithm,
    scenario=result.scenario,
    patient_config=patient_config,
    duration_minutes=240,
    time_step=5,
    output_dir="results/data_sample",
)

outputs["results"].head()


### Recap
You can now go from **official dataset registry → CSV import → runnable scenario** in a few steps.
